In [314]:
from pymongo import MongoClient
from datetime import datetime, timedelta, time
import pytz
# connect to MongoDB
store_id = 3068127015204330700
client = MongoClient('mongodb://localhost:27017/')
db = client['store']
timezone_str = list(db.store_timezone_data.find({"store_id":store_id}))[0].get("timezone_str")
# define the start and end dates for the last 7 days
start_date = (datetime.utcnow() - timedelta(days=57))
# end_date = datetime.utcnow()
end_date = start_date - timedelta(days=1)
# end_date_15 = start_date - timedelta(hours=5)
# end_date = start_date

In [315]:
# 64 - 56

In [316]:
start_date_tz = pytz.utc.localize(start_date).astimezone(pytz.timezone(timezone_str))
end_date_tz = pytz.utc.localize(end_date).astimezone(pytz.timezone(timezone_str))
end_date_tz_15 = pytz.utc.localize(end_date_15).astimezone(pytz.timezone(timezone_str))

In [317]:
str(end_date_15)

'2023-01-25 20:37:19.658582'

In [318]:
end_date_tz_15

datetime.datetime(2023, 1, 25, 15, 37, 19, 658582, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

In [319]:
end_date_tz_15.astimezone(pytz.utc).replace(tzinfo=None)

datetime.datetime(2023, 1, 25, 20, 37, 19, 658582)

In [320]:
end_date_15 = start_date - timedelta(hours=15)
end_date_tz_15 = pytz.utc.localize(end_date_15).astimezone(pytz.timezone(timezone_str))

In [321]:
str(start_date_tz), str(end_date_tz)

('2023-01-25 06:37:58.713834-05:00', '2023-01-24 06:37:58.713834-05:00')

In [322]:
# def get_24_hr(time_str):
#     time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
#     time_str_24hr = time_obj.strftime('%H')
#     return int(time_str_24hr)

In [323]:
import hashlib

In [329]:
pipeline = [
    {
        "$match": {
            "store_id": store_id,
            "timestamp_utc": {"$gte": end_date_tz,  "$lt": start_date_tz},
        }
    },
    {
        "$addFields": {
            "weekday": {"$subtract": [{"$dayOfWeek": "$timestamp_utc"}, 1]},
            'time': { '$dateToString': { 'format': '%H:%M:%S', 'date': "$timestamp_utc" } }
        }
    },
    {
        "$addFields": {
            "filter_field": {
                "$switch": {
                    "branches": [
                        {
                            "case": {
                                "$and": [
                                    {"$eq": ["$weekday", store_hours_data.get("day")]},
                                    {
                                        "$gte": [
                                            "$time", store_hours_data.get("start_time_local"),
                                        ]
                                    },
                                    {
                                        "$lt": [
                                            "$time",store_hours_data.get("end_time_local"),
                                        ]
                                    },
                                ]
                            },
                            "then": hashlib.md5(f'{store_hours_data.get("day")}_{store_hours_data.get("start_time_local")}_{store_hours_data.get("end_time_local")}'.encode('utf-8')).hexdigest(),
                        }
                        for store_hours_data in db.store_hours_data.find(
                            {"store_id": store_id} 
                        )
                    ],
                    "default": False,
                }
            }
        }
    },
    {
        "$match": {
            "filter_field": {"$ne": False},
        }
    },
    { '$sort': { 'timestamp_utc': -1 } },
]

In [330]:
pipeline

[{'$match': {'store_id': 3068127015204330700,
   'timestamp_utc': {'$gte': datetime.datetime(2023, 1, 24, 6, 37, 58, 713834, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
    '$lt': datetime.datetime(2023, 1, 25, 6, 37, 58, 713834, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)}}},
 {'$addFields': {'weekday': {'$subtract': [{'$dayOfWeek': '$timestamp_utc'},
     1]},
   'time': {'$dateToString': {'format': '%H:%M:%S',
     'date': '$timestamp_utc'}}}},
 {'$addFields': {'filter_field': {'$switch': {'branches': [{'case': {'$and': [{'$eq': ['$weekday',
           6]},
         {'$gte': ['$time', '13:00:00']},
         {'$lt': ['$time', '22:00:00']}]},
       'then': 'a4c854a2de786697e1d356b91ac02543'},
      {'case': {'$and': [{'$eq': ['$weekday', 3]},
         {'$gte': ['$time', '12:00:00']},
         {'$lt': ['$time', '23:00:00']}]},
       'then': '241f065d264263e3246a56bf76918477'},
      {'case': {'$and': [{'$eq': ['$weekday', 2]},
         {'$gte': 

In [331]:
results = list(db.store_status_history.aggregate(pipeline))

In [332]:
len(results)

11

In [333]:
uptime_last_hour_ = 0
from itertools import groupby
for k,v in groupby(results,key=lambda x:x['filter_field']):
#     sss = list(v)
    store_active_last_hour = sorted([i['timestamp_utc'] for i in v])
    uptime_last_hour = [(store_active_last_hour[i + 1] - store_active_last_hour[i]).total_seconds()for i in range(len(store_active_last_hour) - 1)]
    uptime_last_hour_ += sum(uptime_last_hour)
#     print(uptime_last_hour_)
print(uptime_last_hour_ / 3600)

9.880536111111109


In [301]:
# uptime_last_hour_ / 3600

In [302]:
# store_active_last_hour = sorted([i['timestamp_utc'] for i in sss])
# uptime_last_hour = [(store_active_last_hour[i + 1] - store_active_last_hour[i]).total_seconds()for i in range(len(store_active_last_hour) - 1)]

In [303]:
# store_active_last_hour

In [304]:
# uptime_last_hour

In [305]:
# results = list(db.store_status_history.aggregate(pipeline))
# results

In [306]:
# pipeline

In [307]:
# results = list(db.store_status_history.aggregate(pipeline))
# print(len(results))
# for doc in results:
#     print(doc)

In [19]:
[[i['day'], i['start_time_local'], i['end_time_local']] for i in list(db.store_hours_data.find(
                            {"store_id": store_id} 
                        ))]

[[6, '13:00:00', '22:00:00'],
 [3, '12:00:00', '23:00:00'],
 [2, '12:00:00', '23:00:00'],
 [0, '12:00:00', '23:00:00'],
 [1, '12:00:00', '23:00:00'],
 [4, '12:00:00', '23:59:59'],
 [5, '12:00:00', '23:59:59']]

In [16]:
store_hours = [dict(t)
               for t in set(
                    tuple(d.items())
                    for d in db.store_hours_data.find({"store_id": store_id} )
                    if not all(val == list(d.values())[0] for val in d.values())
                )
            ]

In [22]:
[[i['day'],i['start_time_local'], i['end_time_local']] for i in store_hours]

[[6, '13:00:00', '22:00:00'],
 [5, '12:00:00', '23:59:59'],
 [3, '12:00:00', '23:00:00'],
 [2, '12:00:00', '23:00:00'],
 [1, '12:00:00', '23:00:00'],
 [0, '12:00:00', '23:00:00'],
 [4, '12:00:00', '23:59:59']]

In [24]:
[i['timestamp_utc'] for i in results]

[datetime.datetime(2023, 1, 25, 12, 3, 13, 553000),
 datetime.datetime(2023, 1, 25, 13, 8, 17, 220000),
 datetime.datetime(2023, 1, 25, 14, 8, 56, 245000),
 datetime.datetime(2023, 1, 25, 15, 8, 11, 801000),
 datetime.datetime(2023, 1, 25, 16, 6, 26, 371000),
 datetime.datetime(2023, 1, 25, 17, 6, 11, 995000),
 datetime.datetime(2023, 1, 25, 18, 1, 21, 34000)]

In [31]:
import datetime

# the list of datetimes
dates = [i['timestamp_utc'] for i in results]

# the schedule
schedule = [[i['day'],i['start_time_local'], i['end_time_local']] for i in store_hours]

# helper function to convert the time string to datetime.time
def parse_time(s):
    return datetime.datetime.strptime(s, '%H:%M:%S').time()

# filter the list of datetimes based on the schedule
filtered_dates = [d for d in dates if d.weekday() == schedule[d.weekday()][0] 
                  and parse_time(schedule[d.weekday()][1]) <= d.time() <= parse_time(schedule[d.weekday()][2])]

# print the filtered list
print(filtered_dates)

[]


In [30]:
dates = [i['timestamp_utc'] for i in results]

ranges = [[i['day'],i['start_time_local'], i['end_time_local']] for i in store_hours]

sublist = []

for date in dates:
    weekday_num = date.weekday()
    for r in ranges:
        if weekday_num == r[0]:
            open_time = datetime.strptime(r[1], '%H:%M:%S').time()
            close_time = datetime.strptime(r[2], '%H:%M:%S').time()
            if open_time <= date.time() <= close_time:
                sublist.append(date)
                break

sublist

[datetime.datetime(2023, 1, 25, 12, 3, 13, 553000),
 datetime.datetime(2023, 1, 25, 13, 8, 17, 220000),
 datetime.datetime(2023, 1, 25, 14, 8, 56, 245000),
 datetime.datetime(2023, 1, 25, 15, 8, 11, 801000),
 datetime.datetime(2023, 1, 25, 16, 6, 26, 371000),
 datetime.datetime(2023, 1, 25, 17, 6, 11, 995000),
 datetime.datetime(2023, 1, 25, 18, 1, 21, 34000)]

In [28]:
sublist

[datetime.datetime(2023, 1, 25, 12, 3, 13, 553000),
 datetime.datetime(2023, 1, 25, 13, 8, 17, 220000),
 datetime.datetime(2023, 1, 25, 14, 8, 56, 245000),
 datetime.datetime(2023, 1, 25, 15, 8, 11, 801000),
 datetime.datetime(2023, 1, 25, 16, 6, 26, 371000),
 datetime.datetime(2023, 1, 25, 17, 6, 11, 995000),
 datetime.datetime(2023, 1, 25, 18, 1, 21, 34000)]

In [66]:
end_date_15

datetime.datetime(2023, 1, 25, 15, 33, 23, 49841)

In [95]:
end_date_15

datetime.datetime(2023, 1, 26, 1, 38, 30, 702340)

In [94]:
end_date_tz_15..strftime('%Y-%m-%d %H:%M:%S.%f')

'2023-01-25 20:38:30.702340'

In [91]:
str(results[0].get("timestamp_utc"))

'2023-01-25 12:03:13.553000'

In [99]:
len([i for i in results if  i.get("timestamp_utc") < end_date_tz_15.astimezone(pytz.utc).replace(tzinfo=None)])

7

In [7]:
# list(db.store_status_history.find({"store_id":store_id}))

In [8]:
# pipeline

In [230]:
pipline = [
    {
        "$group": {
            "_id": 0,
            "maxDate": { "$max": "$timestamp_utc" },
            "minDate": { "$min": "$timestamp_utc" }
        }
    }
]

In [231]:
data = list(db.store_status_history.aggregate(pipline))[0]
maxDate = data.get("maxDate")
minDate = data.get("minDate")

In [232]:
maxDate, minDate

(datetime.datetime(2023, 1, 25, 18, 13, 22, 479000),
 datetime.datetime(2023, 1, 18, 0, 0, 3, 243000))

In [11]:
datetime.utcnow() - minDate

datetime.timedelta(days=64, seconds=22322, microseconds=980194)

In [12]:
results = list(db.store_status_history.aggregate(pipeline))
print(len(results))
for doc in results:
    print(doc)

7
{'_id': ObjectId('641bdd42ed6d5c06464dcbd8'), 'store_id': 3068127015204330700, 'status': 'inactive', 'timestamp_utc': datetime.datetime(2023, 1, 25, 12, 3, 13, 553000), 'weekday': 3, 'time': '12:03:13'}
{'_id': ObjectId('641bdd42ed6d5c06464cde73'), 'store_id': 3068127015204330700, 'status': 'inactive', 'timestamp_utc': datetime.datetime(2023, 1, 25, 13, 8, 17, 220000), 'weekday': 3, 'time': '13:08:17'}
{'_id': ObjectId('641bdd42ed6d5c06464d8c20'), 'store_id': 3068127015204330700, 'status': 'inactive', 'timestamp_utc': datetime.datetime(2023, 1, 25, 14, 8, 56, 245000), 'weekday': 3, 'time': '14:08:56'}
{'_id': ObjectId('641bdd44ed6d5c06465070c4'), 'store_id': 3068127015204330700, 'status': 'inactive', 'timestamp_utc': datetime.datetime(2023, 1, 25, 15, 8, 11, 801000), 'weekday': 3, 'time': '15:08:11'}
{'_id': ObjectId('641bdd42ed6d5c06464b6479'), 'store_id': 3068127015204330700, 'status': 'inactive', 'timestamp_utc': datetime.datetime(2023, 1, 25, 16, 6, 26, 371000), 'weekday': 3, 'ti

In [14]:
end_date

datetime.datetime(2023, 1, 25, 6, 12, 1, 826062)

In [21]:
[i for i in results if end_date  < i.get("timestamp_utc")]

[{'_id': ObjectId('641bdd42ed6d5c06464dcbd8'),
  'store_id': 3068127015204330700,
  'status': 'inactive',
  'timestamp_utc': datetime.datetime(2023, 1, 25, 12, 3, 13, 553000),
  'weekday': 3,
  'time': '12:03:13'},
 {'_id': ObjectId('641bdd42ed6d5c06464cde73'),
  'store_id': 3068127015204330700,
  'status': 'inactive',
  'timestamp_utc': datetime.datetime(2023, 1, 25, 13, 8, 17, 220000),
  'weekday': 3,
  'time': '13:08:17'},
 {'_id': ObjectId('641bdd42ed6d5c06464d8c20'),
  'store_id': 3068127015204330700,
  'status': 'inactive',
  'timestamp_utc': datetime.datetime(2023, 1, 25, 14, 8, 56, 245000),
  'weekday': 3,
  'time': '14:08:56'},
 {'_id': ObjectId('641bdd44ed6d5c06465070c4'),
  'store_id': 3068127015204330700,
  'status': 'inactive',
  'timestamp_utc': datetime.datetime(2023, 1, 25, 15, 8, 11, 801000),
  'weekday': 3,
  'time': '15:08:11'},
 {'_id': ObjectId('641bdd42ed6d5c06464b6479'),
  'store_id': 3068127015204330700,
  'status': 'inactive',
  'timestamp_utc': datetime.dateti

In [399]:
arr =  [i['timestamp_utc'] for i in results]

output = [(arr[i+1] - arr[i]).total_seconds() for i in range(len(arr)-1)]
print(output)

[3903.667, 3639.025, 3555.556, 3494.57, 3585.624, 3309.039]


In [385]:
sum(output) // 60

358.0

In [386]:
(sum(output) / 60) / 24

14.921861805555556

In [88]:
# list(db.store_hours_data.find(
#                             {"store_id": store_id}
#                         ))

In [96]:
# pipeline

In [54]:
# from datetime import datetime, time
# # time_str = '12:10:32'
# def get_24_hr(time_str):
#     time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
#     time_str_24hr = time_obj.strftime('%I')
#     return time_str_24hr

In [55]:
# db.store_hours_data.find_one()

In [56]:
# [{"case": {"$and": [{"$eq": ["$weekday", store_hours_data.get("day")]}, {"$gte": ["$hour", get_24_hr(store_hours_data.get("start_time_local"))]}, {"$lt": ["$hour", get_24_hr(store_hours_data.get("end_time_local"))]}]}, "then": True} for store_hours_data in db.store_hours_data.find({"store_id":store_id})]

In [57]:
start_date

datetime.datetime(2023, 1, 24, 3, 57, 7, 893177)

In [58]:
# run the pipeline and print the results
results = list(db.store_status_history.aggregate(pipeline))
print(len(results))
for doc in results:
    print(doc)

0


In [317]:
results

[{'_id': ObjectId('641b4dae3f79ea11b685679b'),
  'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': datetime.datetime(2023, 1, 24, 11, 9, 42, 559000),
  'weekday': 2,
  'hour': 11},
 {'_id': ObjectId('641b4daa3f79ea11b6810918'),
  'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': datetime.datetime(2023, 1, 24, 12, 9, 5, 147000),
  'weekday': 2,
  'hour': 12},
 {'_id': ObjectId('641b4da93f79ea11b67e819b'),
  'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': datetime.datetime(2023, 1, 24, 13, 7, 58, 448000),
  'weekday': 2,
  'hour': 13},
 {'_id': ObjectId('641b4dab3f79ea11b6815dd6'),
  'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': datetime.datetime(2023, 1, 24, 14, 10, 9, 423000),
  'weekday': 2,
  'hour': 14},
 {'_id': ObjectId('641b4da73f79ea11b67b590d'),
  'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': datetime.datetime(2023, 1, 24, 15, 10, 1, 222000),
  'week

In [97]:
arr =  [i['timestamp_utc'] for i in results]

output = [(arr[i+1] - arr[i]).total_seconds() for i in range(len(arr)-1)]
print(output)

[3562.588, 3533.301, 3730.975, 3591.799, 3460.518, 3573.904, 3933.443, 3416.772, 3650.353, 3611.444, 3691.365, 3415.942, 3476.396, 39735.975, 3529.894, 3545.916, 3901.748, 3306.21, 3762.221, 3697.9, 3039.404]


In [98]:
sum(output) // 60

1852.0

In [99]:
1852.0 //24

77.0

In [240]:
.total_seconds()
[1, 2, 3, 4 , 5]

generate output 1-2, 2-3, 3-4, 4-5
[-1, 1, 1, 1] = 2

[datetime.datetime(2023, 1, 25, 18, 2, 30, 627000),
 datetime.datetime(2023, 1, 25, 17, 11, 51, 223000),
 datetime.datetime(2023, 1, 25, 16, 10, 13, 323000),
 datetime.datetime(2023, 1, 25, 15, 7, 31, 102000),
 datetime.datetime(2023, 1, 25, 14, 12, 24, 892000),
 datetime.datetime(2023, 1, 25, 13, 7, 23, 144000),
 datetime.datetime(2023, 1, 25, 12, 8, 17, 228000),
 datetime.datetime(2023, 1, 25, 11, 9, 27, 334000),
 datetime.datetime(2023, 1, 24, 22, 12, 19, 21000),
 datetime.datetime(2023, 1, 24, 21, 10, 47, 656000),
 datetime.datetime(2023, 1, 24, 20, 10, 36, 212000),
 datetime.datetime(2023, 1, 24, 19, 9, 45, 859000),
 datetime.datetime(2023, 1, 24, 18, 12, 49, 87000),
 datetime.datetime(2023, 1, 24, 17, 7, 15, 644000),
 datetime.datetime(2023, 1, 24, 16, 7, 41, 740000),
 datetime.datetime(2023, 1, 24, 15, 10, 1, 222000),
 datetime.datetime(2023, 1, 24, 14, 10, 9, 423000),
 datetime.datetime(2023, 1, 24, 13, 7, 58, 448000),
 datetime.datetime(2023, 1, 24, 12, 9, 5, 147000),
 datetim

In [ ]:
results = list(db.store_status_history.aggregate(pipeline))

In [111]:
# run the pipeline and print the results
results = list(db.store_status_history.aggregate(pipeline))
print(len(results))
for doc in results:
    print(doc)

37
{'_id': ObjectId('641b4da73f79ea11b67b1523'), 'store_id': 8419537941919820732, 'status': 'active', 'timestamp_utc': datetime.datetime(2023, 1, 24, 9, 0, 43, 697000), 'weekday': 3, 'hour': 9}
{'_id': ObjectId('641b4da73f79ea11b67b7697'), 'store_id': 8419537941919820732, 'status': 'active', 'timestamp_utc': datetime.datetime(2023, 1, 25, 13, 3, 12, 143000), 'weekday': 4, 'hour': 13}
{'_id': ObjectId('641b4da73f79ea11b67bc94c'), 'store_id': 8419537941919820732, 'status': 'active', 'timestamp_utc': datetime.datetime(2023, 1, 24, 5, 54, 30, 441000), 'weekday': 3, 'hour': 5}
{'_id': ObjectId('641b4da83f79ea11b67c5b30'), 'store_id': 8419537941919820732, 'status': 'active', 'timestamp_utc': datetime.datetime(2023, 1, 25, 11, 2, 2, 516000), 'weekday': 4, 'hour': 11}
{'_id': ObjectId('641b4da83f79ea11b67cae0c'), 'store_id': 8419537941919820732, 'status': 'active', 'timestamp_utc': datetime.datetime(2023, 1, 25, 14, 3, 11, 846000), 'weekday': 4, 'hour': 14}
{'_id': ObjectId('641b4da83f79ea11b6